## Sentiment scores generation using Finbert model
 - [Finbert github](https://github.com/ProsusAI/finBERT/tree/master)
 - [Huggingface link](https://huggingface.co/ProsusAI/finbert)

In [31]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch import nn

import numpy as np
import pandas as pd

In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

/Users/junwei/Desktop/FYP Code/sentiment-analysis-volatility-forecasting-1/venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
def generate_sentiment_values(text: str) -> list[float]:
    inputs = tokenizer(text, return_tensors='pt', padding=True)
    logits = model(**inputs)[0]

    # Convert logits to softmax probabilities.
    probabilities = nn.functional.softmax(logits, dim=-1)

    # The probability labels are 'positive', 'negative', 'neutral' (NEED TO CONFIRM)
    return probabilities.detach().numpy().reshape((3,))

### Sense check

In [5]:
test_string = 'Earnings reported are falling'
generate_sentiment_values(test_string)

array([0.02060783, 0.08169658, 0.8976956 ], dtype=float32)

## Finbert on Tesla news


1. Get sentiment for each article

In [26]:
tesla_df = pd.read_csv('../data/Tesla/tesla_gpt_summarised_sentiments.csv')
#df = df[['timestamp','article_url','lead_paragraph','abstract','adjusted_date']]

In [ ]:
tesla_df

,adjusted_date,gpt_summary
0,2019-10-02,"The news of a potential legal battle involving a prominent figure against a political figure could create a sense of uncertainty around the market, potentially leading to negative sentiment surrounding companies involved in similar sectors, such as Tesla. Investors may perceive this as a distraction that could hinder innovation and long-term growth. Increased regulatory scrutiny could also lead to volatility in the tech market, influencing negative sentiment towards related stocks, including Tesla. Consequently, the overall reaction may lean towards pessimism, likely causing a decline in Tesla's stock price."
1,2019-10-03,"This news presents a mixed outlook for Tesla, suggesting a cautiously optimistic scenario. The record quarterly deliveries highlight robust demand, boosting investor confidence and indicating strong operational performance. However, lingering skepticism regarding profit generation causes concern, potentially dampening enthusiasm. Overall, while the positive news of increased deliveries could initially support a favorable stock response, the uncertainty about profits may temper gains, leading to a cautious investor sentiment."
2,2019-10-09,"This news may have a negative impact on Tesla's stock price. The proposed corporate tax rules could complicate Tesla's ability to optimize its tax strategy, potentially increasing its overall tax burden. This added financial strain might deter investors, fearing reduced profitability in the face of heavier taxation. Moreover, the reference to challenges faced by American businesses in foreign markets, exemplified by the N.B.A., adds an additional layer of uncertainty regarding Tesla’s operations and growth potential, particularly in crucial markets like China. Overall, these factors may lead to decreased investor confidence and could negatively influence Tesla's stock performance."
3,2019-10-10,"Negative sentiment surrounding Tesla's actions in response to criticism from China's state media may lead to a decline in its stock price, as investors could perceive the company's decision as a weakness or potential vulnerability in its international operations. The backlash could raise concerns about regulatory challenges in China, which is a significant market for Tesla, potentially harming future sales and profitability. As a result, this development might create uncertainty in the market, further exerting downward pressure on Tesla's stock value."
4,2019-10-11,"This news could negatively impact Tesla's stock price as it highlights the challenges faced by American tech companies, suggesting potential barriers to entry and strained relations in the crucial Chinese market. Difficulties in accessing customers may lead to reduced sales opportunities and weakened growth prospects for Tesla, raising concerns among investors about its ability to capitalize on one of the largest automotive markets globally. Additionally, this precarious situation might prompt uncertainty regarding future collaborations and profitability, further diminishing investor confidence."


In [ ]:
for i, row in tesla_df.iterrows():
    output = generate_sentiment_values(row['gpt_summary'])
    preamble_tesla = "Evaluate the above news on Tesla's stock price."
    combined_str = f"""<news>{row['gpt_summary']}</news> <instructions>{preamble_tesla}</instructions> """    
    output = generate_sentiment_values(combined_str)
    tesla_df.at[i, 'pos_sentiment'] = output[0]
    tesla_df.at[i, 'neg_sentiment'] = output[1]
    tesla_df.at[i, 'neutral_sentiment'] = output[2]

Save to the same file first

In [29]:
tesla_df

,Unnamed: 0,adjusted_date,gpt_summary,pos_sentiment,neg_sentiment,neutral_sentiment
0,394,2019-10-02,"The news of a potential legal battle involving a prominent figure against a political figure could create a sense of uncertainty around the market, potentially leading to negative sentiment surrounding companies involved in similar sectors, such as Tesla. Investors may perceive this as a distraction that could hinder innovation and long-term growth. Increased regulatory scrutiny could also lead to volatility in the tech market, influencing negative sentiment towards related stocks, including Tesla. Consequently, the overall reaction may lean towards pessimism, likely causing a decline in Tesla's stock price.",0.008944,0.961645,0.029411
1,50,2019-10-03,"This news presents a mixed outlook for Tesla, suggesting a cautiously optimistic scenario. The record quarterly deliveries highlight robust demand, boosting investor confidence and indicating strong operational performance. However, lingering skepticism regarding profit generation causes concern, potentially dampening enthusiasm. Overall, while the positive news of increased deliveries could initially support a favorable stock response, the uncertainty about profits may temper gains, leading to a cautious investor sentiment.",0.807424,0.148471,0.044106
2,207,2019-10-09,"This news may have a negative impact on Tesla's stock price. The proposed corporate tax rules could complicate Tesla's ability to optimize its tax strategy, potentially increasing its overall tax burden. This added financial strain might deter investors, fearing reduced profitability in the face of heavier taxation. Moreover, the reference to challenges faced by American businesses in foreign markets, exemplified by the N.B.A., adds an additional layer of uncertainty regarding Tesla’s operations and growth potential, particularly in crucial markets like China. Overall, these factors may lead to decreased investor confidence and could negatively influence Tesla's stock performance.",0.009642,0.966554,0.023804
3,372,2019-10-10,"Negative sentiment surrounding Tesla's actions in response to criticism from China's state media may lead to a decline in its stock price, as investors could perceive the company's decision as a weakness or potential vulnerability in its international operations. The backlash could raise concerns about regulatory challenges in China, which is a significant market for Tesla, potentially harming future sales and profitability. As a result, this development might create uncertainty in the market, further exerting downward pressure on Tesla's stock value.",0.008986,0.966405,0.024610
4,272,2019-10-11,"This news could negatively impact Tesla's stock price as it highlights the challenges faced by American tech companies, suggesting potential barriers to entry and strained relations in the crucial Chinese market. Difficulties in accessing customers may lead to reduced sales opportunities and weakened growth prospects for Tesla, raising concerns among investors about its ability to capitalize on one of the largest automotive markets globally. Additionally, this precarious situation might prompt uncertainty regarding future collaborations and profitability, further diminishing investor confidence.",0.008820,0.967364,0.023817


In [ ]:
tesla_df.to_csv('../data/Tesla/tesla_gpt_summarised_sentiments.csv', index=False)

## Finbert on S&P news


1. Get sentiment for each article

In [38]:
spy_df = pd.read_csv('../data/S&P_market/s&p_gpt_summarised_sentiments.csv')
#df = df[['timestamp','article_url','lead_paragraph','abstract','adjusted_date']]

In [ ]:
spy_df

,adjusted_date,gpt_summary
0,2019-09-30,"The decision by WeWork to postpone its initial public offering signifies a significant setback, which could lead to negative market sentiment, as it reflects deepening concerns about the company's sustainability and viability. Such developments may contribute to a cautious outlook among investors, potentially dampening confidence in high-growth sectors like commercial real estate, and this can negatively impact the S&P price index as broader market sentiments could shift toward pessimism and risk aversion."
1,2019-09-30,"The increase in taxes would negatively impact consumer spending and could lead to a decline in economic activity, ultimately resulting in a bearish sentiment that may pull down the S&P price index as investors react to the potential decrease in growth and profitability for companies reliant on consumer demand."
2,2019-09-30,"This news presents a negative outlook for the S&P price index as it highlights challenges for Seamless and Grubhub, two significant players in the food delivery sector. With restaurant owners expressing dissatisfaction and a city council investigation looming, concerns arise over potential regulatory impacts and a decline in profit margins, which could lead to decreased investor confidence. Additionally, these issues may result in increased operational costs and a potential reduction in market share, further dampening financial performance for companies within this sector."
3,2019-09-30,"This piece of news could have a negative impact on the S&P price index. The mention of failures in major companies like Uber and WeWork underscores a broader concern regarding the sustainability and foundational strength of tech startups, which could lead to decreased investor confidence in the market. Such failures may suggest vulnerabilities in other sectors or similar companies, leading to cautious sentiment among investors and a potential downturn in market performance."
4,2019-09-30,"This news presents a troubling outlook for the S&P price index, as it underscores the potential for government overreach and increased scrutiny of high-profile business figures. The implications of state pressure on influential tycoons like Li Ka-shing signal instability in the market, fostering a climate of uncertainty among investors. Such a narrative could deter investment and lead to heightened volatility, as concerns over regulatory interventions might lead to declining confidence in the broader economic landscape. In sum, the atmosphere of unease surrounding business operations may negatively impact the S&P price index."


In [ ]:
for i, row in spy_df.iterrows():
    output = generate_sentiment_values(row['gpt_summary'])
    preamble_spy = "Evaluate the above news on S&P stock index."
    combined_str = f"""<news>{row['gpt_summary']}</news> <instructions>{preamble_spy}</instructions> """    
    output = generate_sentiment_values(combined_str)
    spy_df.at[i, 'pos_sentiment'] = output[0]
    spy_df.at[i, 'neg_sentiment'] = output[1]
    spy_df.at[i, 'neutral_sentiment'] = output[2]

In [41]:
spy_df

,Unnamed: 0,adjusted_date,gpt_summary,pos_sentiment,neg_sentiment,neutral_sentiment
0,2,2019-09-30,"The decision by WeWork to postpone its initial public offering signifies a significant setback, which could lead to negative market sentiment, as it reflects deepening concerns about the company's sustainability and viability. Such developments may contribute to a cautious outlook among investors, potentially dampening confidence in high-growth sectors like commercial real estate, and this can negatively impact the S&P price index as broader market sentiments could shift toward pessimism and risk aversion.",0.009970,0.959472,0.030558
1,3,2019-09-30,"The increase in taxes would negatively impact consumer spending and could lead to a decline in economic activity, ultimately resulting in a bearish sentiment that may pull down the S&P price index as investors react to the potential decrease in growth and profitability for companies reliant on consumer demand.",0.009024,0.962636,0.028340
2,4,2019-09-30,"This news presents a negative outlook for the S&P price index as it highlights challenges for Seamless and Grubhub, two significant players in the food delivery sector. With restaurant owners expressing dissatisfaction and a city council investigation looming, concerns arise over potential regulatory impacts and a decline in profit margins, which could lead to decreased investor confidence. Additionally, these issues may result in increased operational costs and a potential reduction in market share, further dampening financial performance for companies within this sector.",0.008275,0.970685,0.021040
3,5,2019-09-30,"This piece of news could have a negative impact on the S&P price index. The mention of failures in major companies like Uber and WeWork underscores a broader concern regarding the sustainability and foundational strength of tech startups, which could lead to decreased investor confidence in the market. Such failures may suggest vulnerabilities in other sectors or similar companies, leading to cautious sentiment among investors and a potential downturn in market performance.",0.009357,0.946452,0.044191
4,6,2019-09-30,"This news presents a troubling outlook for the S&P price index, as it underscores the potential for government overreach and increased scrutiny of high-profile business figures. The implications of state pressure on influential tycoons like Li Ka-shing signal instability in the market, fostering a climate of uncertainty among investors. Such a narrative could deter investment and lead to heightened volatility, as concerns over regulatory interventions might lead to declining confidence in the broader economic landscape. In sum, the atmosphere of unease surrounding business operations may negatively impact the S&P price index.",0.010739,0.963194,0.026067
...,...,...,...,...,...,...
12045,12049,2024-09-27,"This piece of news would likely have a positive impact on the S&P price index. The stimulus measures from China signal robust economic support and could enhance global investor confidence, leading to increased trading activity and encouraging capital inflows. As China's stock market experiences strong performance, it may boost sentiment in the U.S. markets as well, facilitating optimism and potential gains in the S&P index.",0.946426,0.014552,0.039022
12046,12040,2024-09-28,"The news about Hurricane Helene indicates significant damage to homes, suggesting a negative impact on the S&P price index. This disaster can lead to increased insurance claims, economic uncertainty, and potential disruptions in the housing market, all of which typically contribute to decreased investor confidence. Consequently, the anticipation of long-term recovery efforts might further dampen market sentiment, leading to a decline in stock prices within the index.",0.008694,0.966439,0.024868
12047,12041,2024-09-28,"This news presents a negative outlook for the S&P price index. OpenAI's unclear financial health raises concerns among inve

In [ ]:
tesla_df.to_csv('../data/S&P_market/s&p_gpt_summarised_sentiments.csv', index=False)